# Introduction
Sales forecasting is one the most common tasks in many sales driven organizations. This activity enables organizations to adequately plan for the future with a degree of confidence. In this tutorial we’ll use Prophet, a package developed by Facebook to show how one can achieve this. This package is available in both Python and R. We assume that the reader has basic understanding of handling time series data in Python. 
  

<p align="center">
  <img width="400" height="300" src="https://media3.giphy.com/media/JtBZm3Getg3dqxK0zP/giphy.gif">
</p>

# Structure

* Downloading Data
* EDA & Data Preprocessing
* Model Fitting
* Making Future Predictions
* Obtaining the Forecasts
* Plotting the Forecasts
* Plotting the Forecast Components
* Cross Validation
* Obtaining the Performance Metrics
* Visualizing Performance Metrics
* Conclusion

# Importing

In [2]:
# fbProphet
!pip install pystan
!pip install prophet
from prophet import Prophet
from prophet.plot import plot_plotly
# Yahoo Finance
!pip install yfinance
import yfinance as yf 
# Others
import pandas as pd
from plotly import graph_objs as go
from datetime import datetime

  Using cached pystan-3.10.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached aiohttp-3.10.10-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached clikit-0.6.2-py2.py3-none-any.whl.metadata (1.6 kB)
INFO: pip is looking at multiple versions of pystan to determine which version is compatible with other requirements. This could take a while.
  Using cached pystan-3.9.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.9.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.8.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached pystan-3.7.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.6.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.5.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pystan-3.4.0-py3-none-any.whl.metadata (3.7 kB)
INFO: pip is still looking at multiple versions of pystan to determine which version is compatible with other requirements. This could take a while.
  Using cached pystan-3.3.0-py3-none-any.whl.metadata

## Selecting a stock

In [3]:
ySymbol="GOOG"

## Downloading Data From yfinance

In [4]:
#for SymbolName in stocksymbols:
data = yf.download(
    # tickers list or string as well
    tickers = ySymbol,
    # use "period" instead of start/end
    # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    # (optional, default is '1mo')
    period = "2y",
    # fetch data by interval (including intraday if period < 60 days)
    # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    # (optional, default is '1d')
    interval = "1d",
    # group by ticker (to access via data['SPY'])
    # (optional, default is 'column')
    group_by = 'ticker',
    # adjust all OHLC(An open-high-low-close chart is a type of chart typically used to illustrate movements in the price of a financial instrument over time) automatically
    # (optional, default is False)
    auto_adjust = True,
    # download pre/post regular market hours data
    # (optional, default is False)
    prepost = True
    )

[*********************100%***********************]  1 of 1 completed


In [5]:
data

Ticker                           GOOG                                      \
Price                            Open        High         Low       Close   
Date                                                                        
2022-10-21 00:00:00+00:00   98.218794  101.371056   97.989361  101.231400   
2022-10-24 00:00:00+00:00  101.839894  102.847422  100.054286  102.717743   
2022-10-25 00:00:00+00:00  103.046939  104.842525  102.767619  104.672943   
2022-10-26 00:00:00+00:00   96.522957   98.298595   94.338320   94.587708   
2022-10-27 00:00:00+00:00   94.078956   94.936850   91.674864   92.373146   
...                               ...         ...         ...         ...   
2024-10-14 00:00:00+00:00  164.910004  167.619995  164.779999  166.350006   
2024-10-15 00:00:00+00:00  167.139999  169.089996  166.050003  166.899994   
2024-10-16 00:00:00+00:00  166.029999  167.279999  165.216003  166.740005   
2024-10-17 00:00:00+00:00  167.380005  167.929993  164.369995  164.509995   
2024-10-18 00:00:00+00:00  164.869995  166.369995  164.750000  165.050003   

Ticker                               
Price                        Volume  
Date                                 
2022-10-21 00:00:00+00:00  28988700  
2022-10-24 00:00:00+00:00  24680800  
2022-10-25 00:00:00+00:00  29910200  
2022-10-26 00:00:00+00:00  71504300  
2022-10-27 00:00:00+00:00  54036500  
...                             ...  
2024-10-14 00:00:00+00:00   9981800  
2024-10-15 00:00:00+00:00  14829300  
2024-10-16 00:00:00+00:00   9968500  
2024-10-17 00:00:00+00:00  15113400  
2024-10-18 00:00:00+00:00  13086900  

[501 rows x 5 columns]

## Resetting Index

In [6]:
data.reset_index(inplace=True)

# EDA & Data Preprocessing

## Plotting the Raw Data

In [8]:
import plotly.offline as py

In [9]:
py.iplot([go.Scatter(
    x=data['Date'],
    y=data['Close']
)])

KeyError: 'Close'

## Fixing the dataset according to FBProphet

In [ ]:
df_train = data[['Date', 'Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})
df_train.head()

# Modelling

[Here](https://www.analyticsvidhya.com/blog/2018/05/generate-accurate-forecasts-facebook-prophet-python-r/) is a link if you want to understand about the model and parameters in detail.

In [ ]:
m = Prophet(daily_seasonality=True)
m.fit(df_train)

## Making Future Predictions

The next step is to prepare our model to make future predictions. This is achieved using the Prophet.make_future_dataframe method and passing the number of days we’d like to predict in the future. We use the periods attribute to specify this. This also include the historical dates. We’ll use these historical dates to compare the predictions with the actual values in the ds column.

* periods: int no. of periods to forecast forward.

In [ ]:
future = m.make_future_dataframe(periods=2*365)

## Obtaining the Forecasts

 
We use the predict method to make future predictions. This will generate a dataframe with a yhat column that will contain the predictions.

In [ ]:
forecast = m.predict(future)

## Plotting the Forecasts

Prophet has an inbuilt feature that enables us to plot the forecasts we just generated. This is achieved using mode.plot() and passing in our forecasts as the argument. The blue line in the graph represents the predicted values while the black dots represents the data in our dataset

In [ ]:
m.plot(forecast)

In [ ]:
py.iplot([
    go.Scatter(x=df_train['ds'], y=df_train['y'], name='Actual'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted')
])

## Plotting the Forecast Components

The plot_components method plots the trend, yearly and weekly seasonality of the time series data.

In [ ]:
#### Visualize Each Components[Trends,yearly]
m.plot_components(forecast)

## Cross Validation

 
Next let’s measure the forecast error using the historical data. We’ll do this by comparing the predicted values with the actual values. In order to perform this operation we select cut of points in the history of the data and fit the model with data upto that cut off point. Afterwards we compare the actual values to the predicted values. The cross_validation method allows us to do this in Prophet. This method take the following parameters as explained below:

* Horizon the forecast horizon.
* Initial the size of the initial training period.
* Period the spacing between cutoff dates.

The output of the cross_validation method is a dataframe containing y the true values and yhat the predicted values. We’ll use this dataframe to compute the prediction errors.

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(m, initial='365 days', period='180 days', horizon = '365 days')
df_cv.head()

## Obtaining the Performance Metrics
 
We use the performance_metrics utility to compute the Mean Squared Error(MSE), Root Mean Squared Error(RMSE),Mean Absolute Error(MAE), Mean Absolute Percentage Error(MAPE) and the coverage of the the yhat_lower and yhat_upper estimates.

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p

## Visualizing Performance Metrics

The performance Metrics can be visualized using the plot_cross_validation_metric utility. Let’s visualize the RMSE below

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='rmse')

# Conclusion

 
As we have seen Prophet is very powerful and effective in time series forecasting.

# Thanks for spending some of your valuable time reading my notebook. If you find any value in here do give an upvote 🎇

# Much Appretiated 🌠 

<p align="center">
  <img width="400" height="300" src="https://media.giphy.com/media/xUPGcg1IJEKGCI6r5e/giphy.gif">
</p>